In [5]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime_cw where tfl_message = 'ok' limit 3
"""
df = pd.read_sql(sql, conn)

,nlc,icscode,station_name,id,tfl_request,tfl_response,querydict,journey_time,tfl_message,london_mainline_station_ics,london_mainline_station_text
0,6846,1000691,White Notley,2553,https://api.tfl.gov.uk/Journey/JourneyResults/...,"{""$type"":""Tfl.Api.Presentation.Entities.Journe...","{""to"": ""E145HP"", ""id"": ""cedea352"", ""key"": ""396...",None,ok,None,None
1,5686,90008069,Worplesdon,2554,https://api.tfl.gov.uk/Journey/JourneyResults/...,"{""$type"":""Tfl.Api.Presentation.Entities.Journe...","{""to"": ""E145HP"", ""id"": ""cedea352"", ""key"": ""396...",None,ok,None,None
2,6551,90007791,Whitwell (Derbys),2555,https://api.tfl.gov.uk/Journey/JourneyResults/...,"{""$type"":""Tfl.Api.Presentation.Entities.Journe...","{""to"": ""E145HP"", ""id"": ""cedea352"", ""key"": ""396...",None,ok,None,None


In [9]:
j1 = df.loc[0,"tfl_response"]

import json
j2 = json.loads(j1)

In [17]:
journeys = j2["journeys"]

In [28]:
journey = sorted(journeys, key = lambda x:  x["duration"])[0]


In [47]:
# Remove any journeys at the end which are not national rail
# Initial assumption will be that all national rail legs are done
def remove_non_national_rail(legs):
    # Pop one at a time 
    while True:
        leg = legs.pop()
        if leg['mode']['id'] == "national-rail":
            break
    legs.append(leg)
    return legs


In [49]:
legs = journey["legs"]
legs = remove_non_national_rail(legs)

In [50]:
def summarise_legs(legs):
    final_string_elems = []
    for leg in legs:
        final_string_elems.append(leg["departurePoint"]["commonName"])
    final_string_elems.append(leg["arrivalPoint"]["commonName"])
    return " -> ".join(final_string_elems)

In [51]:
summarise_legs(legs)

'White Notley Rail Station -> Stratford (London) Rail Station'

In [65]:
from dateutil.parser import parse
def get_total_travel_time(legs):
    dt = parse(legs[0]["departureTime"])
    at = parse(legs[-1]["arrivalTime"])
    td = at-dt
    return td.total_seconds()/60
    
get_total_travel_time(legs)  


44.0

In [77]:
#need lat lng of final location
#need lat lng of initial location 
def lat_lng_from_legs(legs):
    d_lat = legs[0]["departurePoint"]["lat"]
    d_lon = legs[0]["departurePoint"]["lon"]
    a_lat = legs[-1]["arrivalPoint"]["lat"]
    a_lon = legs[-1]["arrivalPoint"]["lon"]
    
    return {"depart":{"lat": d_lat, "lng":d_lon}, "arrive":{"lat":a_lat, "lng":a_lon}}


{'arrive': {'lat': 51.542234259219995, 'lng': -0.0028193822},
 'depart': {'lat': 51.83891012131, 'lng': 0.59586342495}}